In [1]:
from web3 import Web3
import numpy as np
from web3_defi_manage_final import *
from calculation import *

In [2]:
RPC_URL = "https://eth-sepolia.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
explorer_api_key="N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
w3=Web3_Network(RPC_URL, PRIVATE_KEY)

In [3]:
RPC_URL = "https://eth-sepolia.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
w3=Web3_Network(RPC_URL, PRIVATE_KEY)

In [4]:
if w3.is_connected():
    print(f"Connected to {w3.chain_id} network")
    print(f'{w3.get_network_name_from_api()}')
    print(w3.address)
    print(f'{w3.network_gas_mode}')

Connected to 11155111 network
Ethereum Sepolia
0x7868EDEfC36A74eba220C75aCFB05c59a4e55822
EIP-1559


In [5]:
w3.get_position_manager_address("0x1238536071E1c677A632429e3655c799b22cDA52")
w3.get_router_address("0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E")
w3.get_factory_address("0x0227628f3F023bb0B980b67D528571c95c6DaC1c")
w3.get_explorer_api_key("N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M")
w3.get_jitzap_address("0x7cf79E46F62e6BeD7092Bb3fD0ae2d3702f509FF")

In [6]:
abi_cache=ABIRegistry(w3)

In [7]:
_=abi_cache.GET_FACTORY_ABI()
_=abi_cache.GET_ROUTER_ABI()
_=abi_cache.GET_NFPM_ABI()
_=abi_cache.GET_JITZAP_ABI()

Successfully fetched ABI from: 0x0227628f3F023bb0B980b67D528571c95c6DaC1c
Successfully fetched ABI from: 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E
Successfully fetched ABI from: 0x1238536071E1c677A632429e3655c799b22cDA52
Successfully fetched ABI from: 0x7cf79E46F62e6BeD7092Bb3fD0ae2d3702f509FF


In [8]:
pm=PositionManager(w3,abi_cache)
router=SwapRouter(w3,abi_cache)
jitzap=JITZAP(w3,abi_cache)

In [9]:
v3_lps=pm.get_all_positions()

Found 1 Uniswap V3 LP NFTs for address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [10]:
v3_lps[0].get_onchain_balances()

(0, 1163656602753786)

In [11]:
v3_lps[0].is_in_range_extended()

Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5


False

In [12]:
pool=v3_lps[0].get_nft_pool()


In [13]:
v3_lps[0].info()

{'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee': 100,
 'tick_lower': 191095,
 'tick_upper': 191097,
 'liquidity': 825005990548330,
 'tokens_fee0': 0,
 'tokens_fee1': 0}

In [14]:
pool.info()

{'sqrtPriceX96': 1184053995327269470519838032670113,
 'current_tick': 192252,
 'liquidity': 436592103153847,
 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee': 100,
 'tickspacing': 1}

In [15]:
token0,token1=v3_lps[0].get_tokens()
print(token0.token_balance)
print(token1.token_balance)

Successfully fetched ABI from: 0xDa317C1d3E835dD5F1BE459006471aCAA1289068
Successfully fetched ABI from: 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14
390771589
88199992592723187


In [16]:
lt=pool.current_tick-10
ut=pool.current_tick+10
pm.simulate_mint(token0,token1,100,lt,ut,token0.token_balance,token1.token_balance)

{'liquidity': 11725508140251721,
 'amount0_used': 389451427,
 'amount1_used': 88199992592723187}

In [17]:
cp=calculate_raw_price_from_sqrt_price(pool.sqrtPriceX96)
lp=calculate_raw_price_by_tick(lt)
up=calculate_raw_price_by_tick(ut)

In [18]:
calculate_required_amount_and_swap(cp,lp,up,token0.token_balance,token1.token_balance)

{'status': 'Success',
 'final_liquidity_L': 1.174524370573833e+16,
 'ratio_required': 226472383.84073848,
 'amount0_target': 390106923.4852529,
 'amount1_target': 8.83484449144818e+16,
 'swap_details': 'token0 -> token1',
 'swap_amount': 664665.5147470832}

In [19]:
router.simulate_exact_input_single(token0,token1,100,int(23239457.349408567))

5185863490982212

In [20]:
token1.token_balance+5187649558378831

93387642151102018

In [21]:
pm.jit_mint(token0,token1,100,slippage=0.02,step=100)

Action: Preparing to mint new LP position NFT...
Details: Token0: USDC, Token1: WETH, Fee: 100, Amounts Desired: 390771589/88199992592723187, Min Amounts: 382956157/85650263948525888, Ticks: [192151, 192353]
Transaction sent: 060924fc786aba39d256f3fcd1338850d9e0d3af9697ee00f01b045f07e161f6
Confirmed in block 9915702


'060924fc786aba39d256f3fcd1338850d9e0d3af9697ee00f01b045f07e161f6'

In [22]:
router.simulate_exact_input_single(token1,token0,100,1793637)

0